In [1]:
import os
import io
import pandas as pd
from collections import namedtuple, OrderedDict

In [2]:
import tensorflow as tf

from PIL import Image
from object_detection.utils import dataset_util

In [3]:
class_labels = {'bottle': 1, 'person': 2, 'backpack': 3, 'handbag': 4, 
                'umbrella': 5, 'bowl': 6, 'cup': 7, 'clock': 8, 
                'knife': 9, 'spoon': 10, 'fork': 11, 'chair': 12, 
                'potted plant': 13, 'dining table': 14, 'keyboard': 15, 'mouse': 16, 
                'laptop': 17, 'cell phone': 18, 'scissors': 19, 'book': 20, 'monitor': 21}

def create_tf_example(group, path):
    with tf.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size
    
    filename = group.filename.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []
    
    for index, row in group.object.iterrows():
        xmins.append(row['xmin'] / width)
        xmaxs.append(row['xmax'] / width)
        ymins.append(row['ymin'] / height)
        ymaxs.append(row['ymax'] / height)
        classes_text.append(row['class'].encode('utf8'))
        classes.append(class_labels[row['class']])
    
    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(filename),
        'image/source_id': dataset_util.bytes_feature(filename),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example

In [4]:
def split(df, group):
    data = namedtuple('data', ['filename', 'object'])
    gb = df.groupby(group)
    return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]

In [5]:
# load csv files
csv_input1 = 'train_coco.csv'
csv_input2 = 'test_coco.csv'
examples1 = pd.read_csv(csv_input1)
grouped1 = split(examples1, 'file_name')
examples2 = pd.read_csv(csv_input2)
grouped2 = split(examples2, 'file_name')

In [6]:
grouped = grouped1 + grouped2

In [7]:
# shuffle and split into train/val and test
import random
random.seed(42)

random.shuffle(grouped)
test_percent = 0.15
test_group = grouped[:int(len(grouped)*test_percent)]
trainval_group = grouped[int(len(grouped)*test_percent):]

random.shuffle(trainval_group)
val_percent = 0.1
val_group = trainval_group[:int(len(grouped)*val_percent)]
train_group = trainval_group[int(len(grouped)*val_percent):]

print('Number of images in test group: %s' % len(test_group))
print('Number of images in train group: %s' % len(train_group))
print('Number of images in val group: %s' % len(val_group))

Number of images in test group: 13334
Number of images in train group: 66674
Number of images in val group: 8889


In [9]:
output_test_path = 'test.record'
output_val_path = 'val.record'
output_train_path = 'train.record'
path = os.path.join(os.getcwd(), 'images/cooc2017')

writer_test = tf.python_io.TFRecordWriter(output_test_path)
for group in test_group:
    tf_example = create_tf_example(group, path)
    writer_test.write(tf_example.SerializeToString())
writer_test.close()
print('Successfully created the TFRecords: %s' % output_test_path)

writer_val = tf.python_io.TFRecordWriter(output_val_path)
for group in val_group:
    tf_example = create_tf_example(group, path)
    writer_val.write(tf_example.SerializeToString())
writer_val.close()
print('Successfully created the TFRecords: %s' % output_val_path)

writer_train = tf.python_io.TFRecordWriter(output_train_path)
for group in train_group:
    tf_example = create_tf_example(group, path)
    writer_train.write(tf_example.SerializeToString())
writer_train.close()
print('Successfully created the TFRecords: %s' % output_train_path)

Successfully created the TFRecords: test.record
Successfully created the TFRecords: val.record
Successfully created the TFRecords: train.record
